In [1]:
import requests
import json
import time
from datetime import datetime

import sys
import pandas as pd

In [2]:
def check_eqkmsg(mykey, endpoint, tday):
    params = {'ServiceKey':mykey, 'fromTmFc':tday, 'toTmFc':tday,
            'dataType':'json', 'pageNo':1, 'numOfRows':10}
    try:
        r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
    except:
        print('기상청 API 호출에 실패했습니다!')
        sys.exit(1)

    if r.status_code != 200:
        if r.status_code == 401:
            r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
        elif r.status_code == 429:
            time.sleep(1)
            r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
        else:
            print('기상청 API 호출에러 발생!:',r.status_code)

    res = json.loads(r.text).get('response')
    rcode = res.get('header')['resultCode']
    if rcode == '00':
        rescount = res.get('body')['totalCount']
        rpage = (rescount//10)+1
    elif rcode == '03':
        print('오늘의 지진 통보문이 없습니다. ({})'.format(rcode))
        sys.exit(1)
    else:
        return res, rpage

In [3]:
def get_eqkmsg(mykey, endpoint, tday, npage):
    params = {'ServiceKey':mykey, 'fromTmFc':tday, 'toTmFc':tday,
            'dataType':'json', 'pageNo':npage, 'numOfRows':10}
    try:
        r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
    except:
        print('기상청 API 호출에 실패했습니다!')
        sys.exit(1)

    if r.status_code != 200:
        if r.status_code == 401:
            r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
        elif r.status_code == 429:
            time.sleep(1)
            r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
        else:
            print('기상청 API 호출에러 발생!:',r.status_code)

    try:
        res = json.loads(r.text).get('response')
    except:
        print('fail')
        res=None
    return res


In [4]:
def get_eqklist(tday, data):
    raw = data
    eqkdata = raw.get('body').get('items')['item']
    eqk_dict = {}
    eqk_list = []
    for i in range(0, len(eqkdata)):
        pick = eqkdata[i]
        eqk_dict['id'] = tday+'-'+str(raw.get('body')['pageNo'])+str(i)
        eqk_dict['day'] = str(pick.get('tmEqk'))[0:7]
        eqk_dict['eqkt'] = str(pick.get('tmEqk'))[8:14]
        eqk_dict['fct'] = str(pick.get('tmFc'))[8:12]
        eqk_dict['mt'] = pick.get('mt')
        eqk_dict['loc'] = pick.get('loc')
        eqk_dict['lat'] = pick.get('lat')
        eqk_dict['lon'] = pick.get('lon')
        eqk_dict['mapURL'] = pick.get('img')
        eqk_dict['fctype'] = pick.get('fcTp')
        eqk_list.append(eqk_dict)
    time.sleep(1)
    pdcols = eqk_list[0].keys()
    eqk_df = pd.DataFrame(columns=pdcols, data=[eqk_list[i].values() for i in range(0, len(eqk_list))])
    return eqk_df

In [5]:
def main():
    mykey = 'hLgHjx4SRbAp3W6wWAgqvPqZB0dDxQlzAPhuLVz1nCYJ9WUW//16Hzz6vy/0n7rE4kB1kJ3/Ia7vxld3oid6IA=='
    endpoint = 'http://apis.data.go.kr/1360000/EqkInfoService'
    today = str(datetime.today())[0:10].split('-')
    tday =  today[0]+today[1]+today[2]
    npage = 1
    res, rpage = check_eqkmsg(mykey, endpoint, tday)
    
    if rpage != 1:
        eqkdf = pd.DataFrame()
        bind = get_eqklist(tday, res)
        eqkdf = eqkdf.append(bind, ignore_index=True)
        while npage < rpage:
            time.sleep(1)
            npage = npage +1
            res = get_eqkmsg(mykey, endpoint, tday, npage)
            bind = get_eqklist(tday, res)
            eqkdf = eqkdf.append(bind, ignore_index=True)
    else:
        eqkdf = get_eqklist(tday, res)
    eqkdf.to_csv("./csv/{}.csv".format(tday), encoding='utf-8')
    return eqkdf
    

In [6]:
if __name__ == '__main__':
    main()

오늘의 지진 통보문이 없습니다. (03)


SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
